In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import time
import math

In [2]:
df = pd.read_pickle("../data/features/analysis.pickle")

In [3]:
import re
columnNames = list(df)
regexTimes = re.compile(".*Times")
timesColumnNames = list(filter(regexTimes.match, columnNames))
for column in timesColumnNames:
    columnNames.remove(column)
    
regexCosts = re.compile("heuristics.*Costs")
costsColumnNames = list(filter(regexCosts.match, columnNames))
for column in costsColumnNames:
    columnNames.remove(column)

In [5]:
# Drop rows with NA
rowsBefore = df.shape[0]
df = df.dropna()
print("Dropped %d rows due to None values" % (rowsBefore - df.shape[0]))

minCostIndices = df[["heuristics.tabuCosts", "heuristics.simulatedAnnealingCosts", "heuristics.graspCosts", "heuristics.geneticCosts", "heuristics.antColonyCosts"]].idxmin(axis=1)

# Remove all *Times columns
df = df[columnNames]

# Remove name column
df = df.drop(["name"], axis=1)

Dropped 30 rows due to None values


In [6]:
intLabels = LabelEncoder().fit_transform(minCostIndices).reshape(-1, 1)
# 5 values for 5 different heuristics
outputs = OneHotEncoder(sparse=False, n_values=5).fit_transform(intLabels)

inputs = scale(df.astype('float64'),axis=1)

size = df.shape[0]
# Test data is separated in cleaning stage
trainSize = int(size * 0.75)
validSize = size - trainSize

inputsTrain = inputs[0:trainSize]
outputsTrain = outputs[0:trainSize]
intLabelsTrain = intLabels[0:trainSize]

inputsValid = inputs[trainSize:]
outputsValid = outputs[trainSize:]
intLabelsValid = intLabels[trainSize:]

/Users/adam/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [7]:
inputsValid.shape

(157, 38)

In [27]:
epoch_count = 0

def minibatch(batchSize, n, input_data, output_data):
    input_batches = np.empty((math.ceil(n/batchSize), batchSize) + input_data.shape[1:])
    output_batches = np.empty((math.ceil(n/batchSize), batchSize) + output_data.shape[1:])
    
    global epoch_count
    epoch_count += 1
    indexes = np.random.permutation(n)
    i = 0
    batch_i = 0
    input_array = np.zeros((batchSize,) + input_data.shape[1:])
    output_array = np.zeros((batchSize,) + output_data.shape[1:])
    for index in indexes:
        input_array[i] = input_data[index]
        output_array[i] = output_data[index]
        i += 1

        if i >= batchSize:
            input_batches[batch_i] = input_array
            output_batches[batch_i] = output_array
            i = 0
            batch_i += 1
    
    if(n % batchSize != 0):
        input_array[i:] = input_data[0:batchSize - i]
        output_array[i:] = output_data[0:batchSize - i]
        input_batches[batch_i] = input_array
        output_batches[batch_i] = output_array
    
    return (input_batches, output_batches)

In [14]:
EPOCHS = 10000

N1 = trainSize
FEATURE_COUNT = df.shape[1]
LABEL_COUNT = 5

NODES1 = 512
NODES2 = 256

ALPHA = 0.08

BATCH_SIZE = 30

STD = 0.1

In [15]:
# Setup Tensorflow

# Constants
x_train_full = tf.constant(inputsTrain, dtype='float32', shape=[trainSize, FEATURE_COUNT])
y_train_full = tf.constant(outputsTrain, dtype='float32', shape=[trainSize, LABEL_COUNT])

x_valid_full = tf.constant(inputsValid, dtype='float32', shape=[validSize, FEATURE_COUNT])
y_valid_full = tf.constant(outputsValid, dtype='float32', shape=[validSize, LABEL_COUNT])

x_train = tf.placeholder(tf.float32, [BATCH_SIZE, FEATURE_COUNT])
y_train = tf.placeholder(tf.float32, [BATCH_SIZE, LABEL_COUNT])

# Variables
W_input = tf.Variable(tf.truncated_normal([FEATURE_COUNT, NODES1], stddev=STD, seed = 0))
b_input = tf.Variable(tf.truncated_normal([1, NODES1], stddev=STD, seed = 0))

W_hidden = tf.Variable(tf.truncated_normal([NODES1, NODES2], stddev=STD, seed = 0))
b_hidden = tf.Variable(tf.truncated_normal([1, NODES2], stddev=STD, seed = 0))

W_hidden2 = tf.Variable(tf.truncated_normal([NODES2, LABEL_COUNT], stddev=STD, seed = 0))
b_hidden2 = tf.Variable(tf.truncated_normal([1, LABEL_COUNT], stddev=STD, seed = 0))

# Optimization
input_layer = tf.nn.relu(tf.matmul(x_train, W_input) + b_input)

hidden_layer = tf.nn.relu(tf.matmul(input_layer, W_hidden) + b_hidden)
hidden2_layer = tf.matmul(hidden_layer, W_hidden2) + b_hidden2

logits_train_full = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(x_train_full, W_input) + b_input), W_hidden) + b_hidden), W_hidden2) + b_hidden2
logits_valid_full = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(x_valid_full, W_input) + b_input), W_hidden) + b_hidden), W_hidden2) + b_hidden2

L2 = tf.reduce_mean(ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))

CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hidden2_layer, labels = y_train) + ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))

CE_train_full = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits_train_full, labels = y_train_full) + ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))
CE_valid_full = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits_valid_full, labels = y_valid_full) + ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))

optimizer = tf.train.AdamOptimizer().minimize(CE)

y_pred_train = tf.nn.softmax(logits_train_full)
y_pred_valid = tf.nn.softmax(logits_valid_full)

init = tf.global_variables_initializer()

In [16]:
# Initialize TensorFlow
sess = tf.Session()
sess.run(init)

In [17]:
def printStats():
    (ce_train,ce_valid,p_train,p_valid,l2) = sess.run([CE_train_full, CE_valid_full, y_pred_train, y_pred_valid, L2])
    labels_train_pred = oneHotArray[p_train.argmax(axis=1)]
    labels_valid_pred = oneHotArray[p_valid.argmax(axis=1)]
    error_train = 1 - accuracy_score(intLabelsTrain, labels_train_pred)
    error_valid = 1 - accuracy_score(intLabelsValid, labels_valid_pred)
    total_compute_time = (time.time() - t_start)/60
    print('%7d %7d%12.5f%12.5f%12.3f%12.3f%12f%12.1f' % (EPOCHS,epoch_count,ce_train,ce_valid,error_train,error_valid,l2,total_elapsed_time))

In [28]:
# Minimize MSE

train = True

oneHotArray = np.array([0, 1, 2, 3, 4])

total_elapsed_time = 0

ce_time = 0

epoch_count = 0

print('%15s%24s%24s' % (' ','cross-entropy','error-rate'))
print('%15s%12s%12s%12s%12s%12s%12s' % ('epoch','training','validation','training','validation','L2','time (min)'))

while(train):
    batch = minibatch(BATCH_SIZE, N1, inputsTrain, outputsTrain)

    for step in range(batch[0].shape[0]):
        x_batch = batch[0][step]
        y_batch = batch[1][step]
        
        t_start = time.time()
        sess.run([optimizer], feed_dict={x_train:x_batch,y_train:y_batch})
        t_end = time.time()
        
        total_elapsed_time += (t_end - t_start)/60
        
        if t_end - ce_time > 6:
#             (ce) = sess.run(CE_train_full)
#             print("cross-entropy = %f" % (ce))
            printStats()
            
            ce_time = time.time()

            
        if epoch_count >= EPOCHS:
            train = False
            break

print("Finished")
print("Elapsed Time: %f" % (total_elapsed_time))
print("Epoch Count: %d" % (epoch_count))

printStats()

                          cross-entropy              error-rate
          epoch    training  validation    training  validation          L2  time (min)
  10000       1     0.72422     1.30976       0.239       0.427    0.092604         0.0
  10000     301     0.67823     1.13223       0.239       0.427    0.046218         0.1
  10000     604     0.67314     1.16352       0.239       0.427    0.047923         0.2
  10000     905     0.67354     1.15514       0.239       0.427    0.047492         0.3
  10000    1203     0.67218     1.14527       0.239       0.427    0.045163         0.4
  10000    1493     0.67375     1.11937       0.239       0.427    0.044583         0.5
  10000    1776     0.67132     1.17175       0.239       0.427    0.045861         0.6
  10000    2056     0.67228     1.16408       0.239       0.427    0.046610         0.7
  10000    2325     0.67369     1.11384       0.239       0.427    0.043823         0.8
  10000    2606     0.67228     1.14187       0.239     